In [1]:
import pygame
import neat 

import time
import os
import random


pygame 2.5.2 (SDL 2.28.3, Python 3.8.19)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:

pygame.init()
pygame.font.init()
WIN_WIDTH = 500
GEN =0
WIN_HEIGHT = 600
BIRD_IMGS = [pygame.transform.scale2x(pygame.image.load(os.path.join("C:/Users/vishnupriya/OneDrive/Desktop/flappybird/imgs","bird1.png"))),pygame.transform.scale2x(pygame.image.load(os.path.join("C:/Users/vishnupriya/OneDrive/Desktop/flappybird/imgs","bird2.png"))),pygame.transform.scale2x(pygame.image.load(os.path.join("C:/Users/vishnupriya/OneDrive/Desktop/flappybird/imgs","bird3.png")))]

In [3]:
PIPE_IMG = pygame.transform.scale2x(pygame.image.load(os.path.join("C:/Users/vishnupriya/OneDrive/Desktop/flappybird/imgs","pipe.png")))
BASE_IMG = pygame.transform.scale2x(pygame.image.load(os.path.join("C:/Users/vishnupriya/OneDrive/Desktop/flappybird/imgs","base.png")))
BG_IMG = pygame.transform.scale2x(pygame.image.load(os.path.join("C:/Users/vishnupriya/OneDrive/Desktop/flappybird/imgs","bg.png")))

STAT_FONT = pygame.font.SysFont("comicsans",50)

In [4]:
class Bird :
    IMGS = BIRD_IMGS
    MAX_ROTATION = 25
    ROT_VEL = 20
    ANIMATION_TIME = 5
    def __init__(self, x,y):
        self.x = x
        self.y = y
        self.tilt = 0
        self.tick_count = 0
        self.vel=0
        self.height = self.y
        self.img_count =0
        self.img = self.IMGS[0]
    def jump(self):
        self.vel = -10.5
        self.tick_count =0
        self.height = self.y
    def move(self):
        self.tick_count +=1
        d = self.vel *self.tick_count + 1.5*self.tick_count **2 #physics
        if d>=16:
            d =  16
        if d<0:
            d-=2
        self.y = self.y +d
        if d<0 or self.y< self.height +50:
            if self.tilt < self.MAX_ROTATION :
                self.tilt  = self.MAX_ROTATION
        else:
            if self.tilt > -90:
                self.tilt-= self.ROT_VEL
    def draw(self,win):
        self.img_count+=1
        if self.img_count < self.ANIMATION_TIME:
            self.img = self.IMGS[0]
        elif self.img_count < self.ANIMATION_TIME*2:
            self.img = self.IMGS[1]
        elif self.img_count < self.ANIMATION_TIME*3:
            self.img = self.IMGS[2]
        elif self.img_count < self.ANIMATION_TIME*4:
            self.img = self.IMGS[1]
        elif self.img_count == self.ANIMATION_TIME*4 +1:
            self.img = self.IMGS[0]
            self.img_count =0
        if self.tilt <= -80:
            self.img = self.IMGS[1]
            self.img_count = self.ANIMATION_TIME *2

        rotated_image = pygame.transform.rotate(self.img , self.tilt)
        new_rect = rotated_image.get_rect(center = self.img.get_rect(topleft =(self.x , self.y)).center)
        win.blit(rotated_image, new_rect.topleft)
    def get_mask(self):
        return pygame.mask.from_surface(self.img)


class Pipe:
    GAP = 200
    VEL  = 5
    def __init__(self, x):
        self.x = x
        self.height = 0 
        

        self.top =0
        self.bottom =  0
        self.PIPE_TOP = pygame.transform.flip(PIPE_IMG , False, True)
        self.PIPE_BOTTOM = PIPE_IMG
        self.passed =False
        self.set_height()
    def set_height(self):
        self.height = random.randrange(50, 450)
        self.top = self.height - self.PIPE_TOP.get_height()
        self.bottom = self.height +self.GAP

    def move(self):
        self.x -= self.VEL

    def draw(self, win):
        win.blit(self.PIPE_TOP, (self.x,self.top))
        win.blit(self.PIPE_BOTTOM , (self.x, self.bottom))
    def collide(self, bird):
        bird_mask = bird.get_mask()
        top_mask =pygame.mask.from_surface(self.PIPE_TOP)
        bottom_mask = pygame.mask.from_surface(self.PIPE_BOTTOM)
        #calc offset
        top_offset = (self.x - bird.x, self.top - round(bird.y))
        bottom_offset= (self.x - bird.x , self.bottom -round(bird.y))
        b_point = bird_mask.overlap(bottom_mask, bottom_offset) #if they dont colllide it return none
        t_point = bird_mask.overlap(top_mask , top_offset)

        if t_point or b_point:
            return True
        return False
        


class Base:
    VEL = 5
    WIDTH = BASE_IMG.get_width()
    IMG = BASE_IMG

    def __init__(self,y):
        self.y =y
        self.x1 =0
        self.x2 = self.WIDTH
    def move(self):
        self.x1 -= self.VEL
        self.x2 -= self.VEL
        if self.x1 + self.WIDTH <0:
            self.x1 = self.x2+self.WIDTH
        if self.x2 + self.WIDTH <0:
            self.x2 = self.x1 + self.WIDTH


    def draw(self,win):
        win.blit(self.IMG , (self.x1,self.y))
        win.blit(self.IMG , (self.x2,self.y))


    
def draw_window(win, birds, pipes ,base,score ,gen):
    win.blit(BG_IMG, (0,0))
    
    for pipe in pipes :
        pipe.draw(win)
    text = STAT_FONT.render("Score: "+str(score), 1 ,(255,255,255))
    win.blit(text, (WIN_WIDTH -10-text.get_width(), 10))

    text = STAT_FONT.render("Gen: "+str(gen), 1 ,(255,255,255))
    win.blit(text, (10, 10))
    
    base.draw(win)
    for bird in birds:
        bird.draw(win)
    pygame.display.update()
    
def main(genomes, config):   
    global GEN
    GEN+=1
    nets =[]
    ge=[]
    birds = []
    for _, g in genomes:
        net = neat.nn.FeedForwardNetwork.create(g,config)
        nets.append(net)
        birds.append(Bird(230,350))
        g.fitness=0
        ge.append(g)

    
    base = Base(530)
    pipes = [Pipe(500)]
    win = pygame.display.set_mode((WIN_WIDTH,WIN_HEIGHT))
    clock = pygame.time.Clock()

    score =0
    run =True
    while run:
        clock.tick(30)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run =False
                pygame.quit()
                quit()

        pipe_ind =0
        if len(birds)>0:
            if len(pipes) >1 and birds[0].x > pipes[0].x + pipes[0].PIPE_TOP.get_width():
                pipe_ind =1
        else:
            run=False
            break

        for x, bird in enumerate(birds):
            bird.move()
            ge[x].fitness +=0.1

            output = nets[x].activate((bird.y,abs(bird.y - pipes[pipe_ind].height),abs(bird.y -pipes[pipe_ind].bottom)))
            if output[0]>0.5:
                bird.jump()
        
        # bird.move()
        add_pipe= False
        rem =[] #to remove the pipes used
        for pipe in pipes:
            for x,bird in enumerate(birds):
                if pipe.collide(bird):
                    ge[x].fitness -=1
                    birds.pop(x)
                    nets.pop(x)
                    ge.pop(x)
                    
                    
                if not pipe.passed and pipe.x < bird.x:
                    pipe.passed = True
                    add_pipe = True
            if pipe.x + pipe.PIPE_TOP.get_width()<0:
                rem.append(pipe)
            
                
            pipe.move()
        if add_pipe:
            score+=1
            for g in ge:
                g.fitness+=5
            pipes.append(Pipe(500))
        for  r in rem:
            pipes.remove(r)
        for x,bird in enumerate(birds):
            if bird.y + bird.img.get_height() >=530 or bird.y <0:
                birds.pop(x)
                nets.pop(x)
                ge.pop(x)
        #####        
        if score>50 :
            break
        #####
        base.move()
        draw_window(win, birds, pipes , base,score,GEN)
        
    
import pickle

def run(config_path):
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)

    # Run for up to 50 generations.
    winner = p.run(main, 3)

    # Save the winner.
    with open('best_bird.pkl', 'wb') as f:
        pickle.dump(winner, f)
        
def load_best_bird(config_path):
    # Load the configuration.
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)
    
    # Load the winner.
    with open('best_bird.pkl', 'rb') as f:
        winner = pickle.load(f)
    
    # Create a neural network from the winner.
    net = neat.nn.FeedForwardNetwork.create(winner, config)
    
    return net

def play_best_bird(config_path):
    config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                         neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)

    # Load the best bird
    with open('best_bird.pkl', 'rb') as f:
        best_genome = pickle.load(f)

    # Create a neural network from the best genome
    net = neat.nn.FeedForwardNetwork.create(best_genome, config)
    bird = Bird(230, 350)

    base = Base(530)
    pipes = [Pipe(500)]
    win = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))
    clock = pygame.time.Clock()

    score = 0
    run = True
    while run:
        clock.tick(30)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                pygame.quit()
                quit()

        # Use the neural network to control the bird
        output = net.activate((bird.y, abs(bird.y - pipes[0].height), abs(bird.y - pipes[0].bottom)))
        if output[0] > 0.5:
            bird.jump()

        bird.move()
        add_pipe = False
        rem = []
        for pipe in pipes:
            if pipe.collide(bird):
                run = False
                break

            if not pipe.passed and pipe.x < bird.x:
                pipe.passed = True
                add_pipe = True

            if pipe.x + pipe.PIPE_TOP.get_width() < 0:
                rem.append(pipe)

            pipe.move()

        if add_pipe:
            score += 1
            pipes.append(Pipe(500))

        for r in rem:
            pipes.remove(r)

        if bird.y + bird.img.get_height() >= 530 or bird.y < 0:
            run = False

        base.move()
        draw_window(win, [bird], pipes, base, score, GEN)

    print(f"Final Score: {score}")




if __name__ == "__main__":
    # local_dir = os.path.dirname(__file__)
    config_path  = r"C:\Users\vishnupriya\OneDrive\Desktop\flappybird\flappy.txt"
    run(config_path)
    play_best_bird(config_path)
# while (True):
#     bird.move()
        
    


 ****** Running generation 0 ****** 

Population's average fitness: 2.33000 stdev: 1.15087
Best fitness: 3.70000 - size: (1, 3) - species 1 - id 100
Average adjusted fitness: 0.493
Mean genetic distance 1.417, standard deviation 0.525
Population of 100 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0   100      3.7    0.493     0
Total extinctions: 0
Generation time: 3.021 sec

 ****** Running generation 1 ****** 

Population's average fitness: 2.76500 stdev: 1.82326
Best fitness: 15.20000 - size: (1, 2) - species 1 - id 187
Average adjusted fitness: 0.124
Mean genetic distance 1.324, standard deviation 0.508
Population of 100 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1   100     15.2    0.124     0
Total extinctions: 0
Generation time: 5.326 sec (4.173 average)

 ****** Running generation 2 ****** 

Population's average fitness: 6.99600 stdev: